In [1]:
import pandas as pd
import os
import gensim
import nltk as nl
import xlrd
from sklearn.linear_model import LogisticRegression
import openpyxl 


<h2>Tried using gensim models

In [37]:
count = 0
train_X = []
train_Y = []
test_X = []
test_Y = []

for file in os.listdir('./final/'):    
    path = './final/' + file
    wb_obj = openpyxl.load_workbook(path) 
    count += 1
    sheet_obj = wb_obj.active 
    m_row = sheet_obj.max_row 
    
      
    for i in range(1, m_row + 1): 
        cell_obj = sheet_obj.cell(row = i, column = 2)
        cell_obj1 = sheet_obj.cell(row = i, column = 3) 
        
        if count < 57:
                
            if cell_obj1.value is None and cell_obj.value is None:
                pass
            elif cell_obj1.value is None and cell_obj.value is not None:
                train_Y.append(0)
                train_X.append(cell_obj.value)
                
            elif cell_obj1.value is not None:
                train_X.append(cell_obj.value)
                if "major" in cell_obj1.value and "start" in cell_obj1.value: 
                    train_Y.append(1)
                elif "minor" in cell_obj1.value and "start" in cell_obj1.value: 
                    train_Y.append(2)
                elif "off" in cell_obj1.value and "start" in cell_obj1.value: 
                    train_Y.append(3)
                else:
                    train_Y.append(0)
            else:
                pass
        else:
            if cell_obj1.value is None and cell_obj.value is None:
                pass
            elif cell_obj1.value is None and cell_obj.value is not None:
                test_Y.append(0)
                test_X.append(cell_obj.value)
                
            elif cell_obj1.value is not None:
                test_X.append(cell_obj.value)
                
                if "major" in cell_obj1.value and "start" in cell_obj1.value: 
                    test_Y.append(1)
                elif "minor" in cell_obj1.value and "start" in cell_obj1.value: 
                    test_Y.append(2)
                elif "off" in cell_obj1.value and "start" in cell_obj1.value: 
                    test_Y.append(3)
                else:
                    test_Y.append(0)
            else:
                pass

In [35]:
tokens = [nl.word_tokenize(sentences.lower()) for sentences in train_X]


In [4]:
model = gensim.models.Word2Vec(tokens, size=300, min_count=1, workers=4)
print("\n Training the word2vec model...\n")



 Training the word2vec model...



In [5]:
model.train(tokens, total_examples=len(tokens), epochs=1000)

(45226941, 80449000)

In [6]:
model.save("word2vec.model")

In [7]:
from gensim.models import Word2Vec, KeyedVectors   
model.wv.save_word2vec_format('model.bin', binary=True)

In [20]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(model.wv.syn0, train_Y[:4310])



/home/saurabh.ramola/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.
/home/saurabh.ramola/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [22]:
predict = clf.predict(model.wv.syn0[:100, :])
# Calculating the score of the predictions
score = clf.score(model.wv.syn0, train_Y[:4310])
print("\nPrediction word2vec : \n", predict)
print("Score word2vec : \n", score)


Prediction word2vec : 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0]
Score word2vec : 
 0.965661252900232


/home/saurabh.ramola/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.
/home/saurabh.ramola/venv/lib/python3.5/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


<h2> Trying pretrained w2v embeddings

In [4]:
from tabulate import tabulate
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [21]:
X = []
y = []

def create_dataset(flag):
    count = 0
    
    for file in os.listdir('./final/'):    
        path = './final/' + file
        wb_obj = openpyxl.load_workbook(path) 
        count += 1
        sheet_obj = wb_obj.active 
        m_row = sheet_obj.max_row 


        for i in range(1, m_row + 1): 
            cell_obj = sheet_obj.cell(row = i, column = 2)
            cell_obj1 = sheet_obj.cell(row = i, column = 3) 

            if count < 57:

                if cell_obj1.value is None and cell_obj.value is None:
                    pass
                elif cell_obj1.value is None and cell_obj.value is not None:
                    y.append(0)
                    X.append(nl.word_tokenize(cell_obj.value.lower()))

                elif cell_obj1.value is not None:
                    if flag:
                        X.append(nl.word_tokenize(cell_obj.value.lower()))
                        if "major" in cell_obj1.value and "start" in cell_obj1.value: 
                            y.append(1)
                        elif "minor" in cell_obj1.value and "start" in cell_obj1.value: 
                            y.append(1)
                        elif "off" in cell_obj1.value and "start" in cell_obj1.value: 
                            y.append(1)
                        else:
                            y.append(0)
                    else:
                        X.append(nl.word_tokenize(cell_obj.value.lower()))
                        if "major" in cell_obj1.value and "start" in cell_obj1.value: 
                            y.append(1)
                        elif "minor" in cell_obj1.value and "start" in cell_obj1.value: 
                            y.append(1)
                        elif "off" in cell_obj1.value and "start" in cell_obj1.value: 
                            y.append(1)
                        else:
                            y.append(0)
                    
create_dataset(0)
X, y = np.array(X), np.array(y)
print ("total examples %s" % len(y))


total examples 9664


In [24]:
y

array([0, 1, 0, ..., 0, 0, 0])

In [25]:
import numpy as np
with open('./glove.6B.50d.txt', "rb") as lines:
    wvec = {line.split()[0].decode('utf-8'): np.array(line.split()[1:],dtype=np.float32)
               for line in lines}

In [26]:
import struct 

glove_small = {}
all_words = set(w for words in X for w in words)
with open('./glove.6B.50d.txt', "rb") as infile:
    for line in infile:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if (word in all_words):
            nums=np.array(parts[1:], dtype=np.float32)
            glove_small[word] = nums


In [27]:

mult_nb = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("multinomial nb", MultinomialNB())])
bern_nb = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("bernoulli nb", BernoulliNB())])
mult_nb_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("multinomial nb", MultinomialNB())])
bern_nb_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("bernoulli nb", BernoulliNB())])
svc = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("linear svc", SVC(kernel="linear"))])
svc_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("linear svc", SVC(kernel="linear"))])

In [28]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(glove_small))])
        else:
            self.dim=0
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

    
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(glove_small))])
        else:
            self.dim=0
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf, 
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    
        return self
    
    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [29]:
etree_glove_small = Pipeline([("glove vectorizer", MeanEmbeddingVectorizer(glove_small)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])
etree_glove_small_tfidf = Pipeline([("glove vectorizer", TfidfEmbeddingVectorizer(glove_small)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])



In [30]:
all_models = [
    ("mult_nb", mult_nb),
    ("mult_nb_tfidf", mult_nb_tfidf),
    ("bern_nb", bern_nb),
    ("bern_nb_tfidf", bern_nb_tfidf),
    ("svc", svc),
    ("svc_tfidf", svc_tfidf),
    ("glove_small", etree_glove_small),
    ("glove_small_tfidf", etree_glove_small_tfidf),
]


unsorted_scores = [(name, cross_val_score(model, X, y, cv=5).mean()) for name, model in all_models]
scores = sorted(unsorted_scores, key=lambda x: -x[1])


print (tabulate(scores, floatfmt=".4f", headers=("model", 'score')))

model                score
-----------------  -------
mult_nb_tfidf       0.9579
svc_tfidf           0.9579
glove_small         0.9577
glove_small_tfidf   0.9575
mult_nb             0.9565
svc                 0.9496
bern_nb             0.9385
bern_nb_tfidf       0.9385


In [31]:
ax = []
ay = []
def benchmark(model, X, y):
    test_size = 0.1
    scores = []
    X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=test_size)
    scores.append(accuracy_score(model.fit(X_train, y_train).predict(X_test), y_test))
    ax = model.fit(X_train, y_train).predict(X_test)
    ay = y_test
    return np.mean(scores)

In [32]:

table = []
for name, model in all_models:
    table.append({'model': name, 
                  'accuracy': benchmark(model, X, y)})
df = pd.DataFrame(table)

In [33]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.1)

In [34]:
arr = svc.fit(X_train, y_train).predict(X_test)

In [35]:
index = 0
for word in arr:
    if(word):
        print(X_test[index])
        print("Correct ==> ", y_test[index])
        print("Predicted ==> ", word)
        index += 1
    

['we', 'saw', 'about', 'them', 'traveling', 'to', 'san', 'francisco', 'to', 'see', 'whether', 'they', 'wanted', 'ten', 'foot', 'wide', 'or', 'twelve', 'foot', 'wide', 'cars', 'or', 'something', '.']
Correct ==>  0
Predicted ==>  1
['oh', ',', 'you', 'bet', '.']
Correct ==>  0
Predicted ==>  1
['uh-huh', '.']
Correct ==>  0
Predicted ==>  1
['oh', '.']
Correct ==>  0
Predicted ==>  1
['uh-huh', ',']
Correct ==>  0
Predicted ==>  1
['uh-huh', '.']
Correct ==>  0
Predicted ==>  1
['well', ',', 'i', 'think', 'there', "'s", 'a', ',', 'a', 'struggle', 'within', 'the', 'soviet', 'union', '.']
Correct ==>  1
Predicted ==>  1
['i', ',', 'i', 'have', 'a', 'pie', 'crust', 'recipe', 'that', 'i', ',', 'that', 'is', 'unique', ',', 'that', 'has', 'vinegar', 'and', 'egg', 'in', 'it', ',']
Correct ==>  1
Predicted ==>  1
['so', ',', 'we', 'managed', 'to', ',', 'uh', ',', 'figure', 'out', 'how', 'to', 'get', 'at', 'just', 'about', 'everything', 'else', ',', 'but', ',', 'uh', ',', 'but', 'not', ',', 'uh'

In [36]:
y_test

array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,